In [ ]:
# Importing necessary libraries
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from yfinance import download

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# Downloading data
data = download('NIO', start='2020-01-01', end='2023-08-13')
nio_df = data[['Close']]  # Working only with closing prices

# Data normalization
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(nio_df.values)

[*********************100%***********************]  1 of 1 completed


In [ ]:
# Create dataset

# Define the sequence length: This means for predicting the next data point, we will use 'seq_length' previous data points.
seq_length = 30
X, y = [], []

# Loop through the scaled_data starting from 'seq_length' to the end
for i in range(seq_length, len(scaled_data)):
    # For each iteration, get the previous 'seq_length' data points as features
    X.append(scaled_data[i-seq_length:i])
    # The next data point after those 'seq_length' data points is taken as the target variable
    y.append(scaled_data[i])

# Convert the lists into numpy arrays for easier manipulation
X = np.array(X)
y = np.array(y)

# Splitting the dataset into training and testing sets

# Calculate the number of data points that constitute 80% of the dataset
train_size = int(0.8 * len(X))
# Split the data at the calculated index to achieve an 80-20 train-test split
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

# Convert data to PyTorch tensors

# Convert the training and testing data to PyTorch tensors
# We also permute the tensors to match the expected input shape of the model
# Expected shape: (batch_size, num_features, sequence_length)
# Given shape: (batch_size, sequence_length, num_features)
# Using permute to swap the last two dimensions
X_train_tensor = torch.FloatTensor(X_train).permute(0, 2, 1)
y_train_tensor = torch.FloatTensor(y_train)
X_test_tensor = torch.FloatTensor(X_test).permute(0, 2, 1)
y_test_tensor = torch.FloatTensor(y_test)

X_train.shape

(703, 30, 1)

In [ ]:
class MultiLayerCNNLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_lstm_layers, output_dim, kernel_size=3):
        super(MultiLayerCNNLSTM, self).__init__()

        # Store the number of LSTM layers and the hidden dimensionality
        self.num_lstm_layers = num_lstm_layers
        self.hidden_dim = hidden_dim

        # CNN Layers
        # ---------------
        self.cnn_layers = nn.Sequential(
            # First convolution layer. Input: [batch_size, input_dim, seq_length]
            nn.Conv1d(input_dim, 32, kernel_size=kernel_size, padding=1), # Output: [batch_size, 32, seq_length]
            nn.ReLU(), # Activation function

            # Second convolution layer
            nn.Conv1d(32, 64, kernel_size=kernel_size, padding=1), # Output: [batch_size, 64, seq_length]
            nn.ReLU(),

            # Max pooling to reduce the sequence length by half
            nn.MaxPool1d(2), # Output: [batch_size, 64, seq_length/2]

            # Third convolution layer
            nn.Conv1d(64, 128, kernel_size=kernel_size, padding=1), # Output: [batch_size, 128, seq_length/2]
            nn.ReLU(),

            # Another max pooling to reduce sequence length further
            nn.MaxPool1d(2)  # Output: [batch_size, 128, seq_length/4]
        )

        # LSTM Layers
        # ---------------
        # LSTM expects input of shape: [batch_size, seq_length, features].
        # Here, features is 128 (the output channels from the previous CNN layer)
        self.lstm = nn.LSTM(128, hidden_dim, num_lstm_layers, batch_first=True)

        # Fully connected layer to produce the final output
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Pass data through CNN layers
        x = self.cnn_layers(x)

        # Permute tensor dimensions to match the LSTM's expected input shape
        x = x.permute(0, 2, 1)  # Shape: [batch_size, seq_length, features]

        # Initialize LSTM hidden and cell states
        h0 = torch.zeros(self.num_lstm_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)
        c0 = torch.zeros(self.num_lstm_layers, x.size(0), self.hidden_dim).requires_grad_().to(device)

        # Pass data through LSTM layers.
        # Output shape: [batch_size, seq_length, hidden_dim]. We're interested in the last sequence step for the FC layer.
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Pass the output of the last sequence step through the Fully connected layer
        out = self.fc(out[:, -1, :])

        return out.squeeze(-1)


In [ ]:
class ConvBiLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_lstm_layers, output_dim, kernel_size=3):
        super(ConvBiLSTM, self).__init__()

        self.num_lstm_layers = num_lstm_layers
        self.hidden_dim = hidden_dim

        # Multiple CNN layers
        self.cnn_layers = nn.Sequential(
            nn.Conv1d(input_dim, 32, kernel_size=kernel_size, padding=1),
            nn.ReLU(),  # Activation function
            nn.Conv1d(32, 64, kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2),  # Pooling to reduce dimensionality
            nn.Conv1d(64, 128, kernel_size=kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(2)
        )

        # Bi-directional LSTM layers
        self.bilstm = nn.LSTM(128, hidden_dim, num_layers=num_lstm_layers, batch_first=True, bidirectional=True)

        # Fully connected layer
        self.fc = nn.Linear(hidden_dim * 2, output_dim)  # Multiplying by 2 for bidirectional

    def forward(self, x):
        # Pass data through CNN layers
        x = self.cnn_layers(x)

        x = x.permute(0, 2, 1)  # Shape: [batch_size, seq_length, features]

        # Initialize LSTM hidden and cell states
        h0 = torch.zeros(self.num_lstm_layers * 2, x.size(0), self.hidden_dim).requires_grad_().to(device)  # * 2 for bidirectional
        c0 = torch.zeros(self.num_lstm_layers * 2, x.size(0), self.hidden_dim).requires_grad_().to(device)

        # Pass data through BiLSTM layers
        out, (hn, cn) = self.bilstm(x, (h0.detach(), c0.detach()))

        # Pass the output of BiLSTM layer to Fully connected layer
        out = self.fc(out[:, -1, :])

        return out.squeeze(-1)

In [ ]:
# Create the ConvBiLSTM model instance
input_dim = 1  # Assuming 1 feature (close value)
hidden_dim = 64
num_lstm_layers = 2
output_dim = 1
kernel_size = 3

model = ConvBiLSTM(input_dim, hidden_dim, num_lstm_layers, output_dim, kernel_size)
#model = MultiLayerCNNLSTM(input_dim, hidden_dim, num_lstm_layers, output_dim, kernel_size)
model.to(device)  # Move the model to the specified device

# Training
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)
criterion = nn.MSELoss()

num_epochs = 1000
for epoch in range(num_epochs):
    model.train()
    outputs = model(X_train_tensor.to(device))
    loss = criterion(outputs, y_train_tensor.squeeze().to(device))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


Epoch [10/1000], Loss: 0.1020
Epoch [20/1000], Loss: 0.0530
Epoch [30/1000], Loss: 0.0275
Epoch [40/1000], Loss: 0.0093
Epoch [50/1000], Loss: 0.0057
Epoch [60/1000], Loss: 0.0052
Epoch [70/1000], Loss: 0.0044
Epoch [80/1000], Loss: 0.0041
Epoch [90/1000], Loss: 0.0038
Epoch [100/1000], Loss: 0.0036
Epoch [110/1000], Loss: 0.0033
Epoch [120/1000], Loss: 0.0031
Epoch [130/1000], Loss: 0.0028
Epoch [140/1000], Loss: 0.0025
Epoch [150/1000], Loss: 0.0023
Epoch [160/1000], Loss: 0.0020
Epoch [170/1000], Loss: 0.0017
Epoch [180/1000], Loss: 0.0015
Epoch [190/1000], Loss: 0.0013
Epoch [200/1000], Loss: 0.0011
Epoch [210/1000], Loss: 0.0011
Epoch [220/1000], Loss: 0.0010
Epoch [230/1000], Loss: 0.0010
Epoch [240/1000], Loss: 0.0009
Epoch [250/1000], Loss: 0.0009
Epoch [260/1000], Loss: 0.0009
Epoch [270/1000], Loss: 0.0008
Epoch [280/1000], Loss: 0.0008
Epoch [290/1000], Loss: 0.0008
Epoch [300/1000], Loss: 0.0008
Epoch [310/1000], Loss: 0.0008
Epoch [320/1000], Loss: 0.0007
Epoch [330/1000],

In [ ]:
# Evaluation
model.eval()
train_preds = model(X_train_tensor.to(device)).detach().cpu().numpy()
test_preds = model(X_test_tensor.to(device)).detach().cpu().numpy()

# Ensuring that the predictions and actuals have the same shape
train_preds = train_preds.reshape(-1, 1)
test_preds = test_preds.reshape(-1, 1)

# Inverse scaling
train_preds_unscaled = scaler.inverse_transform(train_preds)
test_preds_unscaled = scaler.inverse_transform(test_preds)

# We need the unscaled y_train and y_test for plotting the actuals
y_train_unscaled = scaler.inverse_transform(y_train)
y_test_unscaled = scaler.inverse_transform(y_test)

# Plotting
train_dates = nio_df.index[seq_length:seq_length+len(y_train)]
test_dates = nio_df.index[seq_length+len(y_train):]

fig = go.Figure()

# Actual train prices
fig.add_trace(go.Scatter(x=train_dates, y=y_train_unscaled.flatten(), mode='lines', name='Actual Train Prices'))

# Predicted train prices
fig.add_trace(go.Scatter(x=train_dates, y=train_preds_unscaled.flatten(), mode='lines', name='Predicted Train Prices'))

# Actual test prices
fig.add_trace(go.Scatter(x=test_dates, y=y_test_unscaled.flatten(), mode='lines', name='Actual Test Prices'))

# Predicted test prices
fig.add_trace(go.Scatter(x=test_dates, y=test_preds_unscaled.flatten(), mode='lines', name='Predicted Test Prices'))

fig.update_layout(title='NIO Stock Price Predictions',
                   xaxis_title='Date',
                   yaxis_title='Price')
fig.show()


In [ ]:
# Convert the last sequence from testing data to a PyTorch tensor
# The reshape operation ensures that the input is a 3D tensor: [batch_size, features, sequence_length]
# Here, batch_size=1, features=1, and sequence_length=60 (or seq_length)
# We use unsqueeze to add an additional batch dimension, and permute to switch the feature and sequence dimensions
input_sequence = torch.FloatTensor(X_test[-1]).unsqueeze(0).permute(0, 2, 1).to(device)

# Initialize a list to store predicted values
predictions = []

# Loop over the number of days for which we want to predict
# Here, we're predicting for the next 60 days (approximately 2 months)
for _ in range(60):
    # Use the model to make a prediction based on the current input_sequence
    prediction = model(input_sequence.to(device))

    # Store the predicted value after detaching it from computation graph and converting it back to numpy
    predictions.append(prediction.detach().cpu().numpy()[0])

    # Create a new input sequence for the next prediction
    # This is done by removing the oldest value from the input_sequence and appending the current prediction to it
    # This ensures that the model always has the most recent data when making the next prediction
    new_input = prediction.reshape(1, 1, 1)  # Reshape the prediction to match the input_sequence dimensions
    input_sequence = torch.cat([input_sequence[:, :, 1:], new_input], dim=2)

# If you have scaled the training data before feeding it to the model, then you should use the same scaler
# to reverse the scaling operation and get predictions in the original scale
predictions = scaler.inverse_transform(np.array(predictions).reshape(-1, 1))


In [ ]:
predictions

array([[13.58089 ],
       [13.813116],
       [14.322594],
       [14.68793 ],
       [15.087989],
       [15.329043],
       [15.591273],
       [15.827205],
       [16.1379  ],
       [16.444305],
       [16.77424 ],
       [17.084793],
       [17.411213],
       [17.743298],
       [18.029932],
       [18.278282],
       [18.495428],
       [18.70453 ],
       [18.91947 ],
       [19.136189],
       [19.339579],
       [19.527163],
       [19.704742],
       [19.884619],
       [20.059635],
       [20.228914],
       [20.384836],
       [20.531271],
       [20.66906 ],
       [20.801168],
       [20.92655 ],
       [21.046093],
       [21.15806 ],
       [21.260164],
       [21.35127 ],
       [21.43201 ],
       [21.502962],
       [21.56496 ],
       [21.61784 ],
       [21.662004],
       [21.697529],
       [21.722631],
       [21.737139],
       [21.742138],
       [21.73817 ],
       [21.725338],
       [21.704271],
       [21.676245],
       [21.6418  ],
       [21.600338],


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Create future dates for predictions
last_date = nio_df.index[-1]
future_dates = pd.date_range(last_date + pd.Timedelta(days=1), periods=60, freq='B')  # using business days for future dates

# Plotting the actual data and the future predictions
fig = go.Figure()

# Plot the actual data
fig.add_trace(go.Scatter(x=nio_df.index, y=nio_df['Close'], mode='lines', name='Actual Prices'))

# Plot the future predictions
fig.add_trace(go.Scatter(x=future_dates, y=predictions.flatten(), mode='lines', name='Future Predictions', line=dict(dash='dot')))

fig.update_layout(title='NIO Stock Price Future Predictions',
                   xaxis_title='Date',
                   yaxis_title='Price')

fig.show()
